# Join Statements - Lab

## Introduction

In this lab, you'll practice your knowledge of `JOIN` statements, using various types of joins and various methods for specifying the links between them.

## Objectives

You will be able to:

* Write SQL queries that make use of various types of joins
* Compare and contrast the various types of joins
* Discuss how primary and foreign keys are used in SQL
* Decide and perform whichever type of join is best for retrieving desired data

## CRM ERD

In this lab, you'll use the same customer relationship management (CRM) database that you saw from the previous lesson.
<img src='images/Database-Schema.png' width="600">

## Connecting to the Database
Import the necessary packages and connect to the database `'data.sqlite'`.

In [1]:
!ls

README.md
data.sqlite
images
index.ipynb
venn.png


In [2]:
import pandas as pd
import sqlite3

# Connect to the database
conn = sqlite3.connect('data.sqlite')

# Create a cursor object to execute SQL queries
cursor = conn.cursor()


## Select the names of all employees in Boston 

Hint: join the employees and offices tables. Select the first and last name.

In [3]:
# Select the names of all employees in Boston
# Hint: join the employees and offices tables. Select the first and last name.

# SQL query
query = '''
SELECT e.firstName, e.lastName, o.city
FROM employees AS e
JOIN offices AS o
    USING(officeCode)
WHERE o.city = 'Boston'
'''

# Execute the query and fetch the results into a DataFrame
df = pd.read_sql(query, conn)

# Display the DataFrame
df

,firstName,lastName,city
0,Julie,Firrelli,Boston
1,Steve,Patterson,Boston


## Are there any offices that have zero employees?
Hint: Combine the employees and offices tables and use a group by. Select the office code, city, and number of employees.

In [4]:
# SQL query
query = '''
    SELECT o.officeCode, o.city, COUNT(e.employeeNumber) AS numEmployees
    FROM offices AS o
    LEFT JOIN employees AS e
        USING(officeCode)
    GROUP BY officeCode
    HAVING numEmployees = 0;
'''

# Execute the query and fetch the results into a DataFrame
df = pd.read_sql(query, conn)

# Display the DataFrame
df

,officeCode,city,numEmployees
0,27,Boston,0


## Write 3 questions of your own and answer them

In [5]:
# Answers will vary

# Example question: 
"""
How many customers are there per office?
"""
query = '''
SELECT o.officeCode, o.city, COUNT(c.customerNumber) AS customerCount
FROM offices o
JOIN employees e ON o.officeCode = e.officeCode
JOIN customers c ON e.employeeNumber = c.salesRepEmployeeNumber
GROUP BY o.officeCode, o.city;
'''

df = pd.read_sql(query, conn)

df

,officeCode,city,customerCount
0,1,San Francisco,12
1,2,Boston,12
2,3,NYC,15
3,4,Paris,29
4,5,Tokyo,5
5,6,Sydney,10
6,7,London,17


In [6]:
"""
Which employee has the highest number of sales?
"""

query = '''
SELECT e.firstName || ' ' || e.lastName AS employeeName, COUNT(*) AS salesCount
FROM employees e
JOIN customers c 
    ON e.employeeNumber = c.salesRepEmployeeNumber
JOIN orders o 
    ON c.customerNumber = o.customerNumber
GROUP BY e.employeeNumber
ORDER BY salesCount DESC
LIMIT 1;
'''

df = pd.read_sql(query, conn)

df

,employeeName,salesCount
0,Gerard Hernandez,43


In [7]:
"""
What is the average order amount for the top 5 customer?
"""

query = '''
SELECT c.customerName, AVG(od.priceEach * od.quantityOrdered) AS averageOrderAmount
FROM customers c
JOIN orders o ON c.customerNumber = o.customerNumber
JOIN orderdetails od 
    ON o.orderNumber = od.orderNumber
GROUP BY c.customerNumber
ORDER BY averageOrderAmount DESC
LIMIT 5;
'''

df = pd.read_sql(query, conn)

df

,customerName,averageOrderAmount
0,Super Scale Inc.,4139.920588
1,Mini Caravy,3992.595789
2,Tekni Collectables Inc.,3895.550000
3,Gift Depot Inc.,3816.985200
4,"La Corne D'abondance, Co.",3763.196522


In [8]:
"""
How many products are currently out of stock?
"""
query = '''
SELECT COUNT(*) AS outOfStockCount
FROM products
WHERE quantityInStock <= 0;
'''

df = pd.read_sql(query, conn)

df

,outOfStockCount
0,0


In [9]:
"""
What is the total revenue generated by each product category?
"""
query = '''
SELECT p.productLine, SUM(od.priceEach * od.quantityOrdered) AS totalRevenue
FROM products p
JOIN orderdetails od 
    ON p.productCode = od.productCode
GROUP BY p.productLine;
'''

df = pd.read_sql(query, conn)

df

,productLine,totalRevenue
0,Classic Cars,3853922.49
1,Motorcycles,1121426.12
2,Planes,954637.54
3,Ships,663998.34
4,Trains,188532.92
5,Trucks and Buses,1024113.57
6,Vintage Cars,1797559.63


In [10]:
"""
Which employee has the highest average order amount?
"""

query = '''
SELECT e.firstName || ' ' || e.lastName AS employeeName, AVG(od.priceEach * od.quantityOrdered) AS avgOrderAmount
FROM employees e
JOIN customers c 
    ON e.employeeNumber = c.salesRepEmployeeNumber
JOIN orders o 
    ON c.customerNumber = o.customerNumber
JOIN orderdetails od 
    ON o.orderNumber = od.orderNumber
GROUP BY e.employeeNumber
ORDER BY avgOrderAmount DESC
LIMIT 1;
'''

df = pd.read_sql(query, conn)

df

,employeeName,avgOrderAmount
0,Foon Yue Tseng,3438.117394


In [11]:
"""
What is the total sales amount for customers who have made at least two orders?

"""
query = '''
SELECT c.customerName, (
    SELECT SUM(od.priceEach * od.quantityOrdered)
    FROM orders o
    JOIN orderdetails od ON o.orderNumber = od.orderNumber
    WHERE o.customerNumber = c.customerNumber
) AS totalSalesAmount
FROM customers c
WHERE (
    SELECT COUNT(*)
    FROM orders
    WHERE customerNumber = c.customerNumber
) >= 2;
'''

df = pd.read_sql(query, conn)

df


,customerName,totalSalesAmount
0,Atelier graphique,22314.36
1,Signal Gift Stores,80180.98
2,"Australian Collectors, Co.",180585.07
3,La Rochelle Gifts,158573.12
4,Baane Mini Imports,104224.79
...,...,...
92,Motor Mint Distributors Inc.,77726.59
93,Signal Collectibles Ltd.,42570.37
94,"Double Decker Gift Stores, Ltd",29586.15
95,Diecast Collectables,65541.74


In [12]:
"""
What is the average order amount for customers who have placed an order within the last 30 days?

"""

query = '''
SELECT c.customerName, AVG(od.priceEach * od.quantityOrdered) AS averageOrderAmount
FROM customers c
JOIN orders o ON c.customerNumber = o.customerNumber
JOIN orderdetails od ON o.orderNumber = od.orderNumber
WHERE o.orderDate >= (
    SELECT DATE('now', '-30 days')
)
GROUP BY c.customerNumber;
'''

df = pd.read_sql(query, conn)

df


,customerName,averageOrderAmount


## Level Up 1: Display the names of every individual product that each employee has sold

Hint: You will need to use multiple `JOIN` clauses to connect all the way from employee names to product names.

In [13]:
query = '''
SELECT e.firstName || ' ' || e.lastName AS employeeName, p.productName
FROM employees e
JOIN customers c ON e.employeeNumber = c.salesRepEmployeeNumber
JOIN orders o ON c.customerNumber = o.customerNumber
JOIN orderdetails od ON o.orderNumber = od.orderNumber
JOIN products p ON od.productCode = p.productCode
ORDER BY employeeName;
'''

df = pd.read_sql(query, conn)

df

,employeeName,productName
0,Andy Fixter,1996 Moto Guzzi 1100i
1,Andy Fixter,2003 Harley-Davidson Eagle Drag Bike
2,Andy Fixter,P-51-D Mustang
3,Andy Fixter,1936 Harley Davidson El Knucklehead
4,Andy Fixter,1997 BMW R 1100 S
...,...,...
2991,Steve Patterson,2002 Suzuki XREO
2992,Steve Patterson,1928 Ford Phaeton Deluxe
2993,Steve Patterson,1930 Buick Marquette Phaeton
2994,Steve Patterson,American Airlines: B767-300


## Level Up 2: Display the number of products each employee has sold

Alphabetize the results by employee last name.

Hint: Use the `quantityOrdered` column from `orderDetails`. Also, think about how to group the data when some employees might have the same first or last name.

In [14]:
query = '''
SELECT e.employeeNumber, e.firstName || ' ' || e.lastName AS employeeName, COUNT(DISTINCT od.productCode) AS numProductsSold
FROM employees e
JOIN customers c ON e.employeeNumber = c.salesRepEmployeeNumber
JOIN orders o ON c.customerNumber = o.customerNumber
JOIN orderdetails od ON o.orderNumber = od.orderNumber
GROUP BY e.employeeNumber, e.firstName, e.lastName
ORDER BY e.lastName;
'''

df = pd.read_sql(query, conn)

df

,employeeNumber,employeeName,numProductsSold
0,1337,Loui Bondur,101
1,1501,Larry Bott,97
2,1401,Pamela Castillo,100
3,1188,Julie Firrelli,80
4,1611,Andy Fixter,82
5,1702,Martin Gerard,78
6,1370,Gerard Hernandez,109
7,1165,Leslie Jennings,107
8,1504,Barry Jones,98
9,1612,Peter Marsh,97


## Level Up 3: Display the names employees who have sold more than 200 different products

Hint: this is different from the previous question because the quantity sold doesn't matter, only the number of different products

In [15]:
query = '''
SELECT e.firstName || ' ' || e.lastName AS employeeName
FROM employees e
JOIN customers c ON e.employeeNumber = c.salesRepEmployeeNumber
JOIN orders o ON c.customerNumber = o.customerNumber
JOIN orderdetails od ON o.orderNumber = od.orderNumber
GROUP BY e.employeeNumber, e.firstName, e.lastName
HAVING COUNT(DISTINCT od.productCode) > 200;
'''

df = pd.read_sql(query, conn)

df

,employeeName


## Summary

In [16]:
conn.close()

Congrats! You practiced using join statements and leveraged your foreign keys knowledge!